# <font color=purple><center> Stop right now, thank you very much!
    
### <font color=violet><center>Maria Clara Macêdo Lelis
    
    
<font color=green> Neste notebook iremos utilizar uma MLP feita com `Pytorch` para entender e implementar uma estratégia de parada antecipada. A rede será treinada e testada com o dataset `diamonds` da biblioteca `seaborn`.
    
    
    
<font color=green>O primeito passo é importar todas as bibliotecas a serem utilizadas. Depois carregamos o dataset a ser utilizado e realizamos um split de treino, teste e validação. 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd 
import seaborn as sb

In [2]:
df= sb.load_dataset('diamonds')
df.head(1)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43


In [3]:
TAMANHO_TEST_VAL = 0.3
TAMANHO_VAL = 0.5
SEMENTE_ALEATORIA1 = 135790
SEMENTE_ALEATORIA2 = 135791

indices = df.index
indices_treino, indices_teste_val = train_test_split(indices, test_size=TAMANHO_TEST_VAL, random_state=SEMENTE_ALEATORIA1)

df_treino = df.loc[indices_treino]
df_teste_val = df.loc[indices_teste_val]

atr_treino = df_treino[["depth","table","price","x","y","z"]]
target_treino = df_treino["carat"]

indices = df_teste_val.index
indices_teste, indices_val = train_test_split(indices, test_size=TAMANHO_VAL, random_state=SEMENTE_ALEATORIA2)

df_teste = df.loc[indices_teste]
df_val = df.loc[indices_val]

atr_teste = df_teste[["depth","table","price","x","y","z"]]
target_teste = df_teste["carat"]

atr_val = df_val[["depth","table","price","x","y","z"]]
target_val = df_val["carat"]

<font color=green> Antes de criar a nossa rede precisamos transformar os dataframes do `pandas` em tensores pois o `pytorch` não realiza operações com dataframes.

In [4]:
X_treino_tensor = torch.tensor(atr_treino.values, dtype=torch.float32)
y_treino_tensor = torch.tensor(target_treino.values, dtype=torch.long)

X_val_tensor = torch.tensor(atr_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(target_val.values, dtype=torch.long)

num_classes = len(torch.unique(y_treino_tensor))

<font color=green> Agora podemos utilizar a biblioteca `Pytorch` para definir a nossa rede neural na classe MLP.

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.camadas=nn.Sequential(
            nn.Linear(6, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes))

    def forward(self, x):
        return self.camadas(x)


<font color=green>Então podemos treinar uma rede neural MLP, usando o otimizador SGD e a função de perda CrossEntropyLoss. A cada época, ele calcula a saída da rede, mede o erro (perda), ajusta os pesos com base nesse erro e avalia o desempenho em dados de validação. Se a perda de validação melhorar, o modelo continua; caso contrário, ele conta quantas épocas se passaram sem melhora. Se esse número atingir o limite definido pela variável paciencia, o treinamento é interrompido mais cedo para evitar overfitting.

In [8]:
minha_mlp = MLP()
TAXA_DE_APRENDIZADO=0.001
otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)
num_epocas = 1000
melhor_val_loss = float('inf')
paciencia = 10
epocas_sem_melhora = 0
criterio = torch.nn.CrossEntropyLoss()

for epoca in range(num_epocas):
    minha_mlp.train()
    otimizador.zero_grad()
    saida = minha_mlp(X_treino_tensor)
    perda = criterio(saida, y_treino_tensor)
    perda.backward()
    otimizador.step()
    
    minha_mlp.eval()
    with torch.no_grad():
        val_saida = minha_mlp(X_val_tensor)
        val_perda = criterio(val_saida, y_val_tensor)
    
    if epoca %100==0:
        print(f'Época {epoca}, Val Loss: {val_perda.item():.4f}')
    
    if val_perda.item()<melhor_val_loss:
        melhor_val_loss=val_perda.item()
        epocas_sem_melhora=0
    else:
        epocas_sem_melhora+=1
        if epocas_sem_melhora>=paciencia:
            print("Parando cedo por falta de melhora.")
            break


Época 0, Val Loss: 1142.2720
Parando cedo por falta de melhora.


<font color= green>Podemos observar que esse tipo de abordagem é uma maneira eficiente de treinar uma rede neural evitando overfiting e o gasto computacional com modelos que possuem um desempenho subótimo.
    
    
<font color= green> Referências
    
- Material de aula